# PRÁCTICA 2: Clasificador de tweets

Esta práctica consiste en descargarse tweets de 4 usuarios con amplia actividad, de los cuales 2 han de ser similares y 2 diferentes.
A continuación se debe vectorizar el corpus y entrenar una SVM para realizar la clasificación de los tweets.
Por último se hace análisis de sentimiento.

En la siguiente celda se encuentran los imports necesarios para la ejecución del script.

In [1]:
import os
import time
import json
import datetime
import tweepy
import operator
import random

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

En la siguiente celda se definen constantes globales.

* Los usuarios similares escogidos son los astronautas [Ricky Arnold](https://twitter.com/astro_ricky) y [Victor Glover](https://twitter.com/AstroVicGlover).
Los usuarios diferentes seleccionados son [Dave Mustaine](https://twitter.com/davemustaine), cantante de una banda de rock y [Jack Dorsey](https://twitter.com/jack), empresario tecnológico.

* Para hacer uso de la API de Tweeter se definen la clave y el secreto. Ya que solo se va a hacer uso de características de lectura y no de escritura (publicación de tweets, me gusta, etc.) solo son necesarios estos credenciales.
La librería se ocupará de obtener un _bearer token_.
[[1]](https://developer.twitter.com/en/docs/authentication/oauth-2-0)
[[2]](https://docs.tweepy.org/en/stable/auth_tutorial.html#oauth-2-authentication)

* Unas constantes que definirán el comportamiento a la hora de obtener los tweets.
    * La carpeta donde se guardaran las colecciones de tweets.
    * La cantidad de tweets que se solicitarán a la API en cada llamada. Se establece a 200 que es el maximo indicado en la [documentación](https://developer.twitter.com/en/docs/twitter-api/v1/tweets/timelines/api-reference/get-statuses-user_timeline).

    Se establecen unos límites a la hora de obtener páginas de tweets, para que la descarga no se quede de forma indefinida haciendo llamadas a la API:
    * Como mucho se descargarán 50 páginas. Aunque en la [documentación](https://developer.twitter.com/en/docs/twitter-api/v1/tweets/timelines/api-reference/get-statuses-user_timeline) se indica que el maximo de de tweets que se pueden descargar son 3200 en cada pagina es posible que no se obtengan el numero de tweets solicitados devido a que hayan sido borrados. De esta forma con 50 páginas como máximo hay de sobra para obtener los máximos tweets posibles.
    * Los máximos intentos para descargar una página serán 3, en caso de agotar los intentos se descartará la página y se pasará a la siguiente.
    * En caso de que se descarten 3 páginas consecutivas la descarga terminará.

    Se establecen unos retardos para no pasar el límite de llamadas permitidas por la API:
    * Se incluye un retardo por cada llamada a la API, el límite de solicitudes de obtención de tweets (`lists/statuses`) es de [900 solicitudes por cada 15 minutos](https://developer.twitter.com/en/docs/twitter-api/v1/rate-limits). 15 minutos son 900 segundos por lo que el retardo será de un segundo por cada llamada.
    * En caso de que haya un fallo de página se establece un retardo de 30 segundos para asegurarse de que no se sobrepasa el límite establecido por la API.

* Porcentaje de los tweets que se reservarán para el conjunto de test.

* Una semilla para que los números aleatorios sean los mismos, más adelante será necesario.

In [2]:
similar_users = ["astro_ricky", "AstroVicGlover"]
different_users = ["DaveMustaine", "jack"]

twitter_api_key = ""
twitter_api_secret = ""

collections_dir = "collections"
tweets_page = 200
max_pages = 50
max_page_fail_retries = 3
max_consecutive_pages_failed = 3
request_wait = 1
page_fail_wait = 30

test_set_percentage = 0.3

seed = 1024

En la siguiente celda se definen los atributos que se copiarán de cada uno de los objetos que devuelva la librería para cada uno de los tweets.
La función `copy_attributes` copia los atributos que son primitivos y serializables, en caso de que un atributo no exista saltará una excepción que será capturada.

La función `copy_tweet` copia un objeto de tweet devuelto por la librería a un diccionario de Python, hace uso de la función anterior para copiar los atributos primitivos y copia los atributos que han sido convertidos en objetos por la librería de forma que sean serializables.

In [3]:
tweet_attributes = [
    "display_text_range",
    "entities",
    "favorite_count",
    "full_text",
    "id",
    "retweet_count",
    "truncated"
]

def copy_attributes(dest, src, attributes):
    for attribute in attributes:
        try:
            dest[attribute] = src[attribute]
        except KeyError:
            i = None
            try:
                i = src["id"]
            except KeyError:
                pass
            print("KeyError: {}, {}".format(attribute, i))

def copy_tweet(tweet):
    dest = {}
    copy_attributes(dest, vars(tweet), tweet_attributes)
    dest["author_screen_name"] = tweet.author.screen_name
    dest["created_at_timestamp"] = tweet.created_at.timestamp()
    dest["retweeted_status"] = hasattr(tweet, "retweeted_status")
    if hasattr(tweet, "extended_entities"):
        dest["extended_entities"] = tweet.extended_entities
    return dest

La función `retrieve_comments` en la siguiente celda, hace la descarga de los tweets para un usuario que se le pase por parámetro.
Se tendrán en cuenta los limites y los retardos descritos anteriormente cuando sean necesarios.

A la hora de llamar a la API desde el cliente `twitter_client` se le pasa el parámetro `tweet_mode="extended"` para que obtenga los [tweets completos en vez de estar truncados](https://docs.tweepy.org/en/stable/extended_tweets.html#standard-api-methods), en caso contrario se truncarían por compatibilidad.
También se le pasa el parámetro `count=tweets_page` que indica cuántos tweets se quieren obtener por cada página.

La función devuelve una lista de diccionarios que son copias de los objetos que ha obtenido la librería.

In [4]:
def retrieve_comments(twitter_client, name):

    tweets = []
    page = 1
    page_fail_retries = 0
    consecutive_pages_failed = 0

    print("Retrieving {}".format(name))

    while page < max_pages and consecutive_pages_failed < max_consecutive_pages_failed:

        try:
            statuses = twitter_client.user_timeline(id=name, tweet_mode="extended", count=tweets_page, page=page)
        except tweepy.TweepError:
            statuses = None

        if statuses:
            for status in statuses:
                tweets.append(copy_tweet(status))

            page += 1
            page_fail_retries = 0
            consecutive_pages_failed = 0

            print("Retrieved Tweets: {} (Pages: {}/{})".format(len(tweets), page, max_pages), end="\r")
            time.sleep(request_wait)

        else:

            if page_fail_retries < max_page_fail_retries:
                page_fail_retries += 1

                print("Failed to retrieve page {} attempts {}/{}, sleeping {} seconds".format(
                    page,
                    page_fail_retries,
                    max_page_fail_retries,
                    page_fail_wait))
                print("Retrieved Tweets: {} (Pages: {}/{})".format(len(tweets), page, max_pages), end="\r")

            else:
                consecutive_pages_failed += 1

                print("{} consecutive failed attempts on page {}, skipping page {}, (Consecutive pages failed {}/{}), sleeping {} seconds".format(
                    max_page_fail_retries,
                    page, page,
                    consecutive_pages_failed,
                    max_consecutive_pages_failed,
                    page_fail_wait))
                print("Retrieved Tweets: {} (Pages: {}/{})".format(len(tweets), page, max_pages), end="\r")

                page += 1
                page_fail_retries = 0

            time.sleep(page_fail_wait)

    print()

    return tweets

def get_collection(twitter_client, name):

    tweets = retrieve_comments(twitter_client, name)

    collection = {
        "collection_name": name,
        "tweets": tweets,
        "date": datetime.datetime.now().isoformat()
    }

    return collection

A continuación se define la función `get_collections` a la que se le pasa una lista de nombres de usuarios que se tienen que obtener. Primero se comprueba si la colección de tweets para ese usuario ya ha sido descargada, en tal caso la colección se lee de disco.

En caso contrario se descarga la colección y se guarda en disco en formato JSON para ser leída más tarde en caso de que se vuelva a ejecutar el script.

La función devuelve un diccionario con claves, una por cada usuario que se quiere obtener la colección, y valores otro diccionario que contiene una lista de tweets y algunos campos más como la fecha de descarga de los tweets.

In [5]:
def get_collection_path(collection_dir, name):
    return os.path.join(collection_dir, "{}.json".format(name))

def get_collections(names):

    twitter_client = None
    collections = {}

    os.makedirs(collections_dir, exist_ok=True)

    for collection_name in names:

        collection_filename = get_collection_path(collections_dir, collection_name)

        if os.path.isfile(collection_filename):
            # read collection

            print("Using saved collection: {}".format(collection_filename))
            with open(collection_filename) as collection_file:
                collection = json.load(collection_file)

            print("  Tweets: {}".format(len(collection["tweets"])))

        else:
            # retrieve collection

            if not twitter_client:
                twitter_auth = tweepy.AppAuthHandler(twitter_api_key, twitter_api_secret)
                twitter_client = tweepy.API(twitter_auth)

            collection = get_collection(twitter_client, collection_name)

            print("Saving collection: {}".format(collection_filename))
            with open(collection_filename, "w") as collection_file:
                json.dump(collection, collection_file)

        collections[collection_name] = collection

    return collections

En la siguiente celda se llama a la función anteriormente descrita que cargará las colecciones de tweets en la variable `twitter_collections`.

In [6]:
twitter_collections = get_collections(similar_users + different_users)

Using saved collection: collections/astro_ricky.json
  Tweets: 1993
Using saved collection: collections/AstroVicGlover.json
  Tweets: 3037
Using saved collection: collections/DaveMustaine.json
  Tweets: 3178
Using saved collection: collections/jack.json
  Tweets: 3229


La función `split_training_test` a continuación, se le pasa por parámetro una lista de documentos y un porcentaje de _test_ por el que se dividirá la lista de documentos.
La lista de documentos primero será ordenada por el campo `timestamp` para que los documentos queden ordenados por fecha.
La función devuelve dos valores por un lado una lista que será usada como _training_ y otra lista que será usada como _test_.
Los conjuntos devueltos han sido divididos de forma que en el conjunto de _test_ queden los tweets más recientes.

La función `extract_corpus` toma como parámetro las colecciones de tweets, y de las colecciones, extrae, filtra y divide los documentos en dos conjuntos, simplemente listas de texto.
En la función se descartan los retweets porque no son contenido escrito por el usuario seleccionado, sino que son textos que el usuario ha decidido compartir en su _timeline_.
Por otro lado también se descartan los tweets que después de eliminar el contenido extra que está incluido en el texto se quedan vacíos de texto.

En el texto también se incluyen menciones a otros usuarios, en caso de que el tweet sea una respuesta, o también puede incluir un link a una imagen en caso de que el tweet contenga una imagen.
Los tweets incluyen un campo `display_text_range` en el que se indican el inicio y el final del texto escrito por el usuario.
Es posible que se de que un tweet consista únicamente de una imagen en respuesta a otro tweet y que tras eliminar el contenido extra se quede vacío de texto, estos tweets serán descartados.

El diccionario devuelto por esta función tiene la siguiente estructura:

```
{'colección1': {'test':     [<lista de documentos>],
                'training': [<lista de documentos>]},
 'colección2': {'test':     [<lista de documentos>],
                'training': [<lista de documentos>]},
 'colección3': {'test':     [<lista de documentos>],
                'training': [<lista de documentos>]},
 'colección4': {'test':     [<lista de documentos>],
                'training': [<lista de documentos>]}}
```

In [7]:
def split_training_test(corpus, test_percentage):

    corpus_sorted = list(map(
        operator.itemgetter('text'),
        sorted(corpus, key=operator.itemgetter('timestamp'))
    ))

    training_size = int(len(corpus_sorted) * (1-test_percentage))

    return corpus_sorted[:training_size], corpus_sorted[training_size:]

def extract_corpus(collections):

    corpus_sets = {}

    for collection in collections:

        corpus = []

        for tweet in collections[collection]["tweets"]:
            start = tweet["display_text_range"][0]
            end = tweet["display_text_range"][1]

            if not tweet["retweeted_status"] and (end-start) != 0:
                corpus.append({'text': tweet["full_text"][start:end], 'timestamp': tweet['created_at_timestamp']})

        training, test = split_training_test(corpus, test_set_percentage)
        corpus_sets[collection] = {'training': training, 'test': test}

    return corpus_sets

En la siguiente celda se llama a la función anteriormente descrita que extrae de las colecciones el corpus y lo almacena en la variable `collections_corpus_sets`.

In [8]:
collections_corpus_sets = extract_corpus(twitter_collections)

La función a continuación toma por parámetros el corpus extraído en la función anterior y los nombres de las colecciones que se quieren vectorizar en conjunto, los nombres de las colecciones que se corresponden con los nombres de los usuarios.
En primer lugar se concatenan los corpus de _training_ y _test_ de cada colección, _training_ y _test_ por separado.
También se crean los vectores de etiquetas de cada colección correspondiente, a cada colección se le asigna un entero de forma ascendente.

Una vez concatenados los corpus y creado su correspondiente vector de etiquetas se aleatorizan ambos.
Se aleatorizan usando un objeto `Random()` nuevo y la misma semilla, para cada una de las aleatorizaciones, al ser ambos vectores de la misma longitud, la aleatorización resultante es la misma.

A continuación se crea el objeto vectorizador que será usado para vectorizar el corpus.
Para hacer la vectorización primero se utiliza la función `fit_transform` para vectorizar el corpus del conjunto de _training_ de forma que el objeto vectorizador aprenda los parámetros.
A continuación se vectoriza el corpus del conjunto de _test_ utilizando la función `transform` del mismo objeto vectorizador usado anteriormente de forma que la vectorización de ambos conjuntos sea homogénea.

El diccionario devuelto por esta función tiene la estructura siguiente:

```
{'collections': ['colección1', 'colección2'],
 'label_map':   {0: 'colección1', 1: 'colección2'},
 'test':        {'labels': [<lista de etiquetas>],
                 'vectors': <matriz de vectores>},
 'training':    {'labels': [<lista de etiquetas>],
                 'vectors': <matriz de vectores>}}
```

In [9]:
def vectorize_corpus(corpus, collections):

    vectors = {'collections': collections,
               'label_map': {},
               'training': {'labels': []},
               'test': {'labels': []}}

    training_corpus = []
    test_corpus = []
    label = 0

    for collection in collections:

        training_corpus += corpus[collection]['training']
        test_corpus += corpus[collection]['test']

        vectors['training']['labels'] += [label for _ in range(0, len(corpus[collection]['training']))]
        vectors['test']['labels'] += [label for _ in range(0, len(corpus[collection]['test']))]

        vectors['label_map'][label] = collection
        label += 1

    random.Random(seed).shuffle(training_corpus)
    random.Random(seed).shuffle(vectors['training']['labels'])

    vectorizer = TfidfVectorizer(stop_words='english')
    vectors['training']['vectors'] = vectorizer.fit_transform(training_corpus)
    vectors['test']['vectors'] = vectorizer.transform(test_corpus)

    return vectors

A continuación se vectorizan las colecciones de usuarios similares y diferentes por separado.

In [10]:
similar_users_vectors = vectorize_corpus(collections_corpus_sets, similar_users)

different_users_vectors = vectorize_corpus(collections_corpus_sets, different_users)

En la siguiente celda se definen un par de funciones que dados dos vectores de etiquetas, las etiquetas reales de los documentos (`ground_truth`) y las etiquetas predichas por un modelo (`prediction`), calculan la matriz de confusión y la precisión del modelo respectivamente.

La función `train_predict` se encarga de entrenar un modelo y hacer la predicción sobre el conjunto de _test_.
A la función se le pasa un parámetro `vectors` que contiene los vectores de los corpus y el resto de parámetros son la configuración que se le pasan directamente al modelo de una SVM.
Por último la función imprime la matriz de confusión y la precisión obtenida en el modelo.

In [11]:
def calculate_confusion_matrix(ground_truth, prediction):

    confusion_matrix = [[0, 0], [0, 0]]

    for gt, p in zip(ground_truth, prediction):
        confusion_matrix[gt][p] += 1

    return confusion_matrix

def calculate_accuracy(ground_truth, prediction):

    correct = 0

    for gt, p in zip(ground_truth, prediction):
        if gt == p:
            correct += 1

    return correct / len(ground_truth)

def print_confusion_matrix_accuracy(confusion_matrix, accuracy, label_map):

    print('{:<16}{:<16}{:<16}'.format(' ', label_map[0], label_map[1]))
    print('{:<16}{:<16}{:<16}'.format(label_map[0], confusion_matrix[0][0], confusion_matrix[0][1]))
    print('{:<16}{:<16}{:<16}  Accuracy: {:0.3f}'.format(label_map[1], confusion_matrix[1][0], confusion_matrix[1][1], accuracy))
    print()

def train_predict(vectors, *args, **kwargs):

    print('Configuration:', args, kwargs)

    classifier = svm.SVC(*args, **kwargs)
    classifier.fit(vectors['training']['vectors'], vectors['training']['labels'])
    prediction = classifier.predict(vectors['test']['vectors'])

    confusion_matrix = calculate_confusion_matrix(vectors['test']['labels'], prediction)
    accuracy = calculate_accuracy(vectors['test']['labels'], prediction)

    print_confusion_matrix_accuracy(confusion_matrix, accuracy, vectors['label_map'])

    return confusion_matrix, accuracy

A continuación se entrena y se muestran los resultados con la configuración por defecto.

In [12]:
_ = train_predict(similar_users_vectors)
_ = train_predict(different_users_vectors)

Configuration: () {}
                astro_ricky     AstroVicGlover  
astro_ricky     314             197             
AstroVicGlover  143             463               Accuracy: 0.696

Configuration: () {}
                DaveMustaine    jack            
DaveMustaine    681             138             
jack            126             475               Accuracy: 0.814



Al ver los primeros resultados, como era de esperar en el caso de la clasificación de usuarios diferentes se obtiene un mejor resultado ya que es más fácil diferenciarlos.

A continuación se prueban diferentes kernels.

In [13]:
_ = train_predict(similar_users_vectors, kernel='linear')
_ = train_predict(similar_users_vectors, kernel='poly')
_ = train_predict(similar_users_vectors, kernel='sigmoid')

_ = train_predict(different_users_vectors, kernel='linear')
_ = train_predict(different_users_vectors, kernel='poly')
_ = train_predict(different_users_vectors, kernel='sigmoid')

Configuration: () {'kernel': 'linear'}
                astro_ricky     AstroVicGlover  
astro_ricky     331             180             
AstroVicGlover  133             473               Accuracy: 0.720

Configuration: () {'kernel': 'poly'}
                astro_ricky     AstroVicGlover  
astro_ricky     116             395             
AstroVicGlover  87              519               Accuracy: 0.568

Configuration: () {'kernel': 'sigmoid'}
                astro_ricky     AstroVicGlover  
astro_ricky     321             190             
AstroVicGlover  134             472               Accuracy: 0.710

Configuration: () {'kernel': 'linear'}
                DaveMustaine    jack            
DaveMustaine    652             167             
jack            98              503               Accuracy: 0.813

Configuration: () {'kernel': 'poly'}
                DaveMustaine    jack            
DaveMustaine    740             79              
jack            409             192               

Como se puede ver en los resultados cambiar el kernel no ha beneficiado en el caso de usuarios diferentes que se obtiene prácticamente el mismo resultado, 81 % precisión.
En cuanto a los usuarios similares si se ha obtenido una ligera mejora subiendo la precisión al 72 %.
Respecto de la prueba inicial.

A continuación se prueban diferentes valores de C, con el kernel por defecto 'rbf'.

In [14]:

_ = train_predict(similar_users_vectors, C=0.1)
_ = train_predict(similar_users_vectors, C=0.5)
_ = train_predict(similar_users_vectors, C=5)
_ = train_predict(similar_users_vectors, C=10)
_ = train_predict(similar_users_vectors, C=50)

_ = train_predict(different_users_vectors, C=0.1)
_ = train_predict(different_users_vectors, C=0.5)
_ = train_predict(different_users_vectors, C=5)
_ = train_predict(different_users_vectors, C=10)
_ = train_predict(different_users_vectors, C=50)


Configuration: () {'C': 0.1}
                astro_ricky     AstroVicGlover  
astro_ricky     0               511             
AstroVicGlover  0               606               Accuracy: 0.543

Configuration: () {'C': 0.5}
                astro_ricky     AstroVicGlover  
astro_ricky     157             354             
AstroVicGlover  85              521               Accuracy: 0.607

Configuration: () {'C': 5}
                astro_ricky     AstroVicGlover  
astro_ricky     341             170             
AstroVicGlover  172             434               Accuracy: 0.694

Configuration: () {'C': 10}
                astro_ricky     AstroVicGlover  
astro_ricky     341             170             
AstroVicGlover  173             433               Accuracy: 0.693

Configuration: () {'C': 50}
                astro_ricky     AstroVicGlover  
astro_ricky     341             170             
AstroVicGlover  173             433               Accuracy: 0.693

Configuration: () {'C': 0.1}
     

En este caso tampoco se ha obtenido ninguna mejora.
Se puede observar cómo para valores pequeños de C tiende a clasificar todos los documentos a una clase.

A continuación se prueban diferentes valores de C, con el kernel 'linear'.

In [15]:
_ = train_predict(similar_users_vectors, kernel='linear', C=0.1)
_ = train_predict(similar_users_vectors, kernel='linear', C=0.5)
_ = train_predict(similar_users_vectors, kernel='linear', C=5)
_ = train_predict(similar_users_vectors, kernel='linear', C=10)
_ = train_predict(similar_users_vectors, kernel='linear', C=50)

_ = train_predict(different_users_vectors, kernel='linear', C=0.1)
_ = train_predict(different_users_vectors, kernel='linear', C=0.5)
_ = train_predict(different_users_vectors, kernel='linear', C=5)
_ = train_predict(different_users_vectors, kernel='linear', C=10)
_ = train_predict(different_users_vectors, kernel='linear', C=50)

Configuration: () {'kernel': 'linear', 'C': 0.1}
                astro_ricky     AstroVicGlover  
astro_ricky     10              501             
AstroVicGlover  4               602               Accuracy: 0.548

Configuration: () {'kernel': 'linear', 'C': 0.5}
                astro_ricky     AstroVicGlover  
astro_ricky     294             217             
AstroVicGlover  106             500               Accuracy: 0.711

Configuration: () {'kernel': 'linear', 'C': 5}
                astro_ricky     AstroVicGlover  
astro_ricky     330             181             
AstroVicGlover  190             416               Accuracy: 0.668

Configuration: () {'kernel': 'linear', 'C': 10}
                astro_ricky     AstroVicGlover  
astro_ricky     329             182             
AstroVicGlover  198             408               Accuracy: 0.660

Configuration: () {'kernel': 'linear', 'C': 50}
                astro_ricky     AstroVicGlover  
astro_ricky     310             201             
A

A continuación se prueban diferentes grados para el kernel 'poly'.

In [16]:
_ = train_predict(similar_users_vectors, kernel='poly', degree=2)
_ = train_predict(similar_users_vectors, kernel='poly', degree=4)
_ = train_predict(similar_users_vectors, kernel='poly', degree=5)


_ = train_predict(different_users_vectors, kernel='poly', degree=2)
_ = train_predict(different_users_vectors, kernel='poly', degree=4)
_ = train_predict(different_users_vectors, kernel='poly', degree=5)

Configuration: () {'kernel': 'poly', 'degree': 2}
                astro_ricky     AstroVicGlover  
astro_ricky     287             224             
AstroVicGlover  150             456               Accuracy: 0.665

Configuration: () {'kernel': 'poly', 'degree': 4}
                astro_ricky     AstroVicGlover  
astro_ricky     34              477             
AstroVicGlover  60              546               Accuracy: 0.519

Configuration: () {'kernel': 'poly', 'degree': 5}
                astro_ricky     AstroVicGlover  
astro_ricky     17              494             
AstroVicGlover  53              553               Accuracy: 0.510

Configuration: () {'kernel': 'poly', 'degree': 2}
                DaveMustaine    jack            
DaveMustaine    725             94              
jack            246             355               Accuracy: 0.761

Configuration: () {'kernel': 'poly', 'degree': 4}
                DaveMustaine    jack            
DaveMustaine    756             63       

En las últimas pruebas no se han obtenido mejoras, en el caso del kernel 'poly' la precisión ha llegado a caer hasta el 51% que indica que la SVM no ha funcionado bien.

Los mejores resultados obtenidos han sido: para el caso de usuarios similares, el kernel 'linear' con valor de C de '1', obteniendo un 72% de precisión y para el caso de usuarios diferentes, con la configuración por defecto, es decir kernel 'rbf' y C '1'.

A continuación se realiza el apartado opcional _análisis básico de sentimiento_.

En primer lugar se instancia un objeto `SentimentIntensityAnalyzer()` que se usará para analizar todos los documentos de todos los corpus.
En diccionario `collections_scores` se almacenarán los resultados del análisis para calcular la media al final.

In [17]:
analyzer = SentimentIntensityAnalyzer()
collections_scores = {}

for collection in collections_corpus_sets:

    collections_scores[collection] = []
    print('================== {:<16} =================='.format(collection))

    for document in collections_corpus_sets[collection]['training'] + collections_corpus_sets[collection]['test']:

        scores = analyzer.polarity_scores(document)
        collections_scores[collection].append(scores)
        print('pos: {:.2f}, neu: {:.2f}, neg: {:.2f}, compound: {: .2f}  {}'.format(scores['pos'],
                                                                                    scores['neu'],
                                                                                    scores['neg'],
                                                                                    scores['compound'],
                                                                                    document.replace('\n', ' ')))

================== astro_ricky      ==================
pos: 0.21, neu: 0.79, neg: 0.00, compound:  0.46  @CDickerson_PFTP astro photos from #ISS that might interest you + @players4planet http://t.co/LukTkuDssu
pos: 0.00, neu: 1.00, neg: 0.00, compound:  0.00  Heading to the Cape! http://t.co/Lwf6ywJCBq
pos: 0.00, neu: 0.84, neg: 0.16, compound: -0.38  Worked on #orion capsule today figuring out how to get out in an emergency. http://t.co/KPe0xW8x0o
pos: 0.29, neu: 0.71, neg: 0.00, compound:  0.50  Cockpit of NASA's Guppy in El Paso, TX http://t.co/8qlX1ABg2e
pos: 0.26, neu: 0.74, neg: 0.00, compound:  0.72  One of my favorite places on Earth # Chesapeake Bay from my favorite place away from Earth #ISS. http://t.co/SzrRLGJoVT
pos: 0.47, neu: 0.53, neg: 0.00, compound:  0.66  @Roni_McG it was a great day!
pos: 0.00, neu: 1.00, neg: 0.00, compound:  0.00  Flag Day 2013. http://t.co/NB3G107nQP
pos: 0.00, neu: 1.00, neg: 0.00, compound:  0.00  At the Neutral Buoyancy Lab (NBL) today-where w

Mostrar la media de cada una de las puntuaciones para cada una de las colecciones.

In [18]:
print('{:<16}{:<8}{:<8}{:<8}{:<8}'.format('Averages', 'pos', 'neu', 'neg', 'compound'))

for collection in collections_scores:
    print('{:<16}{:<8.2f}{:<8.2f}{:<8.2f}{:<8.2f}'.format(
        collection,
        sum(map(lambda s: s['pos'], collections_scores[collection])) / len(collections_scores[collection]),
        sum(map(lambda s: s['neu'], collections_scores[collection])) / len(collections_scores[collection]),
        sum(map(lambda s: s['neg'], collections_scores[collection])) / len(collections_scores[collection]),
        sum(map(lambda s: s['compound'], collections_scores[collection])) / len(collections_scores[collection])
    ))

Averages        pos     neu     neg     compound
astro_ricky     0.20    0.78    0.02    0.32    
AstroVicGlover  0.26    0.72    0.02    0.39    
DaveMustaine    0.23    0.73    0.05    0.29    
jack            0.20    0.76    0.04    0.22    
